# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps 
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_path = 'Output_data/cities.csv'
# Import the CSV into a pandas DataFrame
cities_df = pd.read_csv(weather_path)
cities_df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cape Town,-33.9258,18.4232,56.10,61,20,5.75,ZA,1661708397
1,1,Hargeisa,9.5600,44.0650,75.02,50,75,15.79,SO,1661708397
2,2,Canary Islands,28.0000,-15.5000,71.56,50,20,27.63,ES,1661708398
3,3,Pangnirtung,66.1451,-65.7125,50.00,57,75,9.22,CA,1661708398
4,4,Tuktoyaktuk,69.4541,-133.0374,42.80,93,100,5.75,CA,1661708399
...,...,...,...,...,...,...,...,...,...,...
574,574,Price,39.5994,-110.8107,79.12,35,0,6.91,US,1661708836
575,575,Tocache,-8.1842,-76.5125,90.59,42,53,2.84,PE,1661708837
576,576,Manavālakurichi,8.1333,77.3000,77.20,86,28,12.33,IN,1661708837
577,577,Arzgir,45.3728,44.2206,78.31,35,2,18.43,RU,1661708838


In [3]:
# Delete the extra index column 
del cities_df['Unnamed: 0']
cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cape Town,-33.9258,18.4232,56.10,61,20,5.75,ZA,1661708397
1,Hargeisa,9.5600,44.0650,75.02,50,75,15.79,SO,1661708397
2,Canary Islands,28.0000,-15.5000,71.56,50,20,27.63,ES,1661708398
3,Pangnirtung,66.1451,-65.7125,50.00,57,75,9.22,CA,1661708398
4,Tuktoyaktuk,69.4541,-133.0374,42.80,93,100,5.75,CA,1661708399
...,...,...,...,...,...,...,...,...,...
574,Price,39.5994,-110.8107,79.12,35,0,6.91,US,1661708836
575,Tocache,-8.1842,-76.5125,90.59,42,53,2.84,PE,1661708837
576,Manavālakurichi,8.1333,77.3000,77.20,86,28,12.33,IN,1661708837
577,Arzgir,45.3728,44.2206,78.31,35,2,18.43,RU,1661708838


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_df[['Lat', 'Lon']]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities_df['Humidity']),
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

NameError: name 'locations' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.


Note: Feel free to adjust your specifications, but make sure to limit the number of rows returned by your API requests to a reasonable number.

In [5]:
perfect_weather_cities = cities_df.loc[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80 )
                                      & (cities_df['Humidity'] < 40) & (cities_df['Cloudiness'] == 0)
                                                  & (cities_df['Wind Speed'] < 10)]
perfect_weather_cities

NameError: name 'cities_df' is not defined

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Define hotel_df to store the hotel names 
hotel_df = perfect_weather_cities

# params dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key }

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # Base url 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    response = requests.get(base_url, params=params)
    response = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] =  response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")  

NameError: name 'perfect_weather_cities' is not defined

In [7]:
# Display the hotel DataFrame with the hotel names 
hotel_df

NameError: name 'hotel_df' is not defined

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

NameError: name 'hotel_df' is not defined

In [9]:
hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
markers = gmaps.marker_layer(locations)
# Add the layers to the map
fig.add_layer(hotel_layer)
fig.add_layer(markers)
fig

NameError: name 'locations' is not defined